### Question 1: 1-way ANOVA.
Mechanical parts such as computer disk drives, light bulbs, and glue bonds eventually fail.  One experimental method for reliability is called an accelerated life test.
Parts under stress will usually fail sooner than parts that are unstressed. By modeling the lifetimes of parts under various stresses, we can estimate (extrapolate
to) the lifetime of parts that are unstressed. That way we get an estimate of the unstressed lifetime without having to wait the complete unstressed lifetime.

Nelson (1990) gave an example where the goal was to estimate the lifetime (in hours) of an encapsulating resin for gold-aluminum bonds in integrated circuits operating at 120$^{o}C$. Since the lifetimes were expected to be
rather long, an accelerated test was used. 

Thirty-seven units were assigned
at random to one of five different temperature stresses, ranging from 175$^{o}C$ to
250$^{o}C$. The following table gives the $\log_{10}$ lifetimes in hours for the test units:


|Temperature |$\log_{10}$ times till failure |   
|--- | ------|
|175 | 2.04, 1.85, 1.91, 1.96, 2.00, 1.88, 1.92, 1.90 | 
|194 | 1.66, 1.66, 1.71, 1.61, 1.42, 1.55, 1.76, 1.66| 
|213 | 1.53, 1.35, 1.54, 1.27, 1.38, 1.26, 1.31, 1.38 | 
|231 | 1.15, 1.21, 1.22, 1.28, 1.17, 1.17, 1.16 |
|250 | 1.26, 1.02, 0.83, 1.09, 1.08, 1.06 |


(1) Visualize the data from the experiment using box-plot.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
data_175=[2.04, 1.85, 1.91, 1.96, 2.00, 1.88, 1.92, 1.90]
data_194=[1.66, 1.66, 1.71, 1.61, 1.42, 1.55, 1.76, 1.66]
data_213=[1.53, 1.35, 1.54, 1.27, 1.38, 1.26, 1.31, 1.38]
data_231=[1.15, 1.21, 1.22, 1.28, 1.17, 1.17, 1.16]
data_250=[1.26, 1.02, 0.83, 1.09, 1.08, 1.06]
y=sum([data_175, data_194, data_213,data_231, data_250],[])
factor=sum([['175']*len(data_175), ['194']*len(data_194), ['213']*len(data_213),['231']*len(data_231), ['250']*len(data_250)],[])
df=pd.DataFrame({'resp':y, 'Trt':factor})

(2) Conduct 1-way ANOVA.  Do you reject the null hypothesis that all the treatment means are the same?

**Ans**:

In [2]:
## code here

(3) Verify if the three model assumptions are valid.

**Ans**:

In [3]:
## code here

### Question 2: 2-way ANOVA 

The following table shows the results of an experiment on sprouting barley.  The 30 lots of seeds are divided at random into ten groups of three lots each, with each group receiving a different treatment. The ten treatments
are the factorial combinations of amount of `water` used for sprouting (factor A) with two levels, and `age` of the seeds (factor B) with five levels. The response measured is the `number` of seeds sprouting.

         
|ml H2O ||Age of|  Seeds |(weeks)||
|--- |--- |--- |--- |--- |--- 
|    |1 | 3 |6 | 9| 12| 
|  4  |11,9,6 | 7,16,17 |9,19,35 | 13,35,28| 20,37,45| 
|  8  |8,3,3 | 1,7,3 |5,9,9 | 1,10,9| 11,15,25| 



(1) Use interaction plot to check if there is an interaction effect between `water` used and `age` of seeds.\
**Ans**:

In [4]:
## code here
y=[11,9,6, 7,16,17, 9,19,35, 13,35,28, 20,37,45, 8,3,3, 1,7,3, 5,9,9,  1,10,9,  11,15,25]
water=sum([['4']*15,['8']*15],[])
age=sum([['1']*3,['3']*3,['6']*3,['9']*3,['12']*3]*2,[])
df=pd.DataFrame({'resp':y, 'water':water,'age':age} )

(2) Conduct ANOVA for the experiment and draw your conclusions.\
**Ans**:

In [5]:
## code here

(3) Suppose there is no interaction effect between `water` used and `age` of seeds, use Tukey’s method to conduct the post-hoc analysis among the five levels of `age` of the seeds and visualize the multiple comparisons at significance
level 0.05. Write down your conclusions (explain how many groups are formed).\
**Ans**:

In [6]:
## code here